# train, vaildation and test set 🛠
<p>
    모델 평가의 핵심은 데이터를 train, vaildation과 test로 구분하는 것이다.<br>
    이렇게하는 주된 이유는 test set은 평가할 때 딱 한번만 사용하기 위해서다.<br>
    보통 학습할 때 그 정보를 바탕으로 hyperparameter를 튜닝하게 되는데, 이때<br>
    imformation leak가 발생한다. 따라서 타당한 평가를 위해서 데이터셋을 구분<br>
    하는것은 매우 중요하다.
</p>
<br/>

## set vaildation data from train data 🔨
<p>
    train data를 나누는 것은 간단해보이지만, 데이터가 적을 때 매우 큰 효율을 야기한다.
</p>
<br/>종류
<ul>
    <li>hold-out vaildation</li>
    <li>k-fold cross-vaildation</li>
    <li>iterated k-fold cross-vaildation</li>
</ul>

### hold-out vaildation ⚽
<p>
    데이터의 일정량을 검증 세트로 떼어 놓는 방식이다. 남은 데이터에서 <br>
    훈련하고 검증 세트로 평가를 한다. 정보 누설을 피하기위해 검증 세트<br>
    는 따로 떼어 놓는다.
</p>

In [ ]:
# hold-out vaildation 구현 예시
import numpy as np

data = np.linspace(1,1000000,num=100000000)

num_vaildation_samples = 1000

np.random.shuffle(data) # 데이터를 섞는 것이 일반적으로 좋음

validation_data = data[:num_vaildation_samples] # 검증 데이터를 생성함
data = data[num_vaildation_samples:]

training_data = data[:] # 훈련세트를 생성함

"""
model = get_model()
model.train(training_data)
vaildation_score = model.evaluate(vaildation_data) # 검증 데이터로 평가함

#모델 튜닝 작업
#훈련, 평가 튜닝 반복 ...

model = get_model()
model.train(np.concatenate([train_data,         # hyperparameter의 튜닝 작업이 끝나면, 
                            vaildation_data]))  # 다시 모든 데이터를 통해 훈련시킴
test_score = model.evaluate(test_data)          # test data를 통해 최종 평가를 함
"""

<p>
    이 방법은 단순하기에 한 가지 단점이 있다. 데이터가 매우 적을때에는 <br>
    검증 세트와 테스트 세트가 전체 데이터의 통계적인 대표를 못할 수 있다.<br>
</p>
<br/>

### k-fold cross-vaildation ⚾
<p>
    hold-out의 단점을 보안하고자 나온 방식이다. 주어진 데이터를 k개로 분할<br>
    한다. 각 fold에서 i번째 데이터는 vaildation으로 이용하고 k-1 데이터들은<br>
    훈련에 사용하는 방식이다. 최종 평가는 이렇게 얻은 k개의 점수를 평균을 <br>
    통해 얻는다.
</p>
<img src="./k_fold.png">


In [ ]:
k=4
num_validation_samples = len(data) // k

np.random.shuffle(data)
validation_scores = []

for fold in range(k):
    """
    train data로부터 인덱싱 하는 과정을 
    주의 깊게 봐보자
    """
    validation_data = data[num_validation_samples * fold:
        num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] + \
        data[num_validation_samples * (fold + 1):]
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)

validation_score = np.average(validation_scores)
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

### ITERATED K-FOLD VALIDATION WITH SHUFFLING 🥎
<p>
    비교적 가용 데이터가 적고 가능한 정확하게 모델을 평가하고자<br>
    할 때 사용하는 방법이다. 이 방법은 k-fold를 여러번 적용하되<br>
    k개 분할 전 데이터를 shufffling하는 방법이다. </br>
    최종 점수는 모든 k-겹 교차 검증을 실행해 얻은 점수의 평균이다<br>
    결국 P * K개의 (P는 반복 횟수)의 모델을 훈련하고 평가하기에 <br>
    cost가 비싸다.
</p>

# Things to keep in mind! 📜
<ul>
    <li>Data representativeness : 꼭 무작위로 데이터를 섞고 사용하자. 데이터가 일련으로 정렬된 경우 편향된 학습을 할 수 있다.</li>
    <li>The arrow of time : 한 예로 과거로부터 미래를 예측하려면, 데이터를 분할하기전 무작위로 섞어서는 절대 안된다! 방향이 섞이면 정보 누설이 생긴다. 시간의 방향은 항시 주의히자!</li>
    <li>Redundancy in your data : 주어진 데이터에 중복 데이터가 포함돼있을 수도 있다. 분할 전에 중복 데이터를 제거해주자! 만약 분할 후 중복 데이터가 각각 훈련과 검증 데이터에 들어가면 최악의 경우가 된다!</li>
</ul>